In [ ]:
from ortools.sat.python import cp_model
from hypothesis import given, assume, note, strategies as st
import networkx as nx
from functools import partial
from collections import Counter

In [ ]:
def clear_trades(num_courses, num_students, desired_num_courses, course_preferences, allocated_courses, max_cycle_length):
    def dfs(n, cycle_length):
        

In [ ]:
@given(st.data())
def test_trades_reach_pareto_optimal(data):
    """
    This function uses the Hypothesis library to test the property
    that any allocation of courses to students and configuration of student
    preferences can reach a pareto optimal allocation via clearing mutually
    beneficial trade cycles (one by one).
    Hypothesis uses a combination of random sampling and (if a counterexample is found)
    shrinking of inputs, to either support the proposed hypothesis with testing
    or find a concise counterexample.
    """

    # ============= Setting up Hypothesis strategies and configuration =============
    
    num_courses = data.draw(st.integers(1, 1000), label="Number of courses")
    num_students = data.draw(st.integers(1, 500), label="Number of students")

    # Draws a list mapping [0..size-1] to elements drawn from `strat`
    draw_list = lambda size, strat, label: data.draw(st.lists(strat, min_size=size, max_size=size), label)
    # Draws a list mapping course [0..num_courses-1] to elements drawn from `strat`
    draw_courses_map = partial(draw_list, size=num_courses)
    # Draws a list mapping student [0..num_students-1] to elements drawn from `strat`
    draw_student_map = partial(draw_list, size=num_students)

    # List mapping course to max enrollment
    max_enrollment = draw_courses_map(st.integer(10, 240), label="Course -> max enrollment")

    # List mapping student to desired number of courses
    desired_num_courses = draw_student_map(st.integer(3, 7), label="Student -> desired num courses")

    # Hypothesis strategy for drawing a course
    course_strat = st.integers(min_value=0, max_value=num_courses-1)

    # List mapping student to (potentially incomplete) list of course preferences
    course_preferences = draw_student_map(st.lists(course_strat, max_size=40, unique=True),
        label="Student -> partial course preferences")
    # List mapping student to initially allocated courses
    allocated_courses = draw_student_map(st.sets(course_strat, max_size=6),
        label="Student -> initially allocated courses")
    
    # Assume we haven't violated the max enrollment constraint already
    assume(all(
        enrollment <= max_enrollment[course]
        for course, enrollment in Counter(allocated_courses).items()))
    
    # ============= Setting up Google ORTools constraint programming model =============

    clear_trades(
        num_courses, num_students, desired_num_courses, course_preferences, allocated_courses,
        max_cycle_length=None
    )

    

In [ ]:
# Hypothesis strategies for the total number of courses, and for a single course
num_courses_strat = st.integers(1, 20_000)
course_strat = num_courses_strat.flatmap(lambda num_courses:
    st.integers(min_value=0, max_value=num_courses-1))

# Hypothesis strategy for the total number of students
num_students_strat = st.integers(1, 6000)

# Allocated courses strategy for a single student
course_preferences = num_courses_strat.flatmap(lambda num_courses:
    st.lists(course_strat, max_size=40, unique=True)
allocated_courses = num_courses_strat.flatmap(lambda num_courses:
    st.sets(course_strat, max_size=6)

num_courses.flatmap(lambda n: st.)

students = num_students.flatmap(lambda n: st.lists())

# Maps student [0..num_students-1] to list of allocated courses
allocated_courses = [
    st.lists(st.integers(min_value=0, max_value=num_courses-1), unique=True)
    for _ in range(num_)
]

desired_courses = st.lists(st.integers(min_value=0, max_value=num_courses-1), unique=True)



st.integers(4, 7)

In [ ]:
@st.composite
def allocation_builder(draw,
                  node_data=st.fixed_dictionaries({}),
                  edge_data=st.fixed_dictionaries({}),
                  node_keys=None,
                  min_nodes=0, max_nodes=25,
                  min_edges=0, max_edges=None,
                  graph_type=nx.Graph,
                  self_loops=False, connected=True):
    """
    A :mod:`hypothesis` strategy for building networkx graphs.

    Parameters
    ----------
    draw
        For internal hypothesis use.
    node_data: `hypothesis.SearchStrategy[dict]`
        The strategy to use to generate node attributes. Must generate a
        mapping.
    edge_data: `hypothesis.SearchStrategy[dict]`
        The strategy to use to generate edge attributes. Must generate a
        mapping.
    node_keys: `hypothesis.SearchStrategy[collections.abc.Hashable]` or None
        The strategy to use to generate node keys. Must generate a Hashable. If
        `None`, node keys will be taken from range(0, number_of_nodes).
    min_nodes: int
        The minimum number of nodes that should be in the generated graph. Must
        be positive.
    max_nodes: int or None
        The maximum number of nodes that should be in the generated graph. Must
        be larger than `min_nodes`. `None` means no upper limit.
    min_edges: int
        The minimum number of edges that should be in the generated graph. Less
        edges may be added if the produced graph contains too few nodes.
    max_edges: int or None
        The maximum number of edges that should be in the generated graph.
        `None` means no upper limit. Note that if `connected` is True more edges
        may be added.
    graph_type: class
        The type of graph that should be created.
    self_loops: bool
        Whether self loops (edges between a node and itself) are allowed.
    connected: bool
        If `True`, the generated graph is guaranteed to be a single (weakly)
        connected component.

    Raises
    ------
    ValueError
        - If `min_nodes` < 0.
        - If `max_nodes` < `min_nodes`
        - If the graph has to be connected, but `max_edges` is too small
          relative to `max_nodes`.

    Returns
    -------
    networkx.Graph
        The created graph. The actual type is determined by the argument
        `graph_type`.
    """
    if min_nodes < 0:
        raise ValueError('min_nodes can not be negative')
    if max_nodes is not None and min_nodes > max_nodes:
        raise ValueError('min_nodes must be less than or equal to max_nodes')
    if max_nodes is not None and max_edges is not None and connected and max_edges < max_nodes-1:
        raise ValueError("It's impossible to create a connected graph of {}"
                         "nodes with less than {} edges".format(max_nodes, max_nodes-1))

    graph = graph_type()
    is_multigraph = graph.is_multigraph()
    is_directed = graph.is_directed()

    # Draw node indices and their associated data
    node_datas = draw(st.lists(node_data, min_size=min_nodes, max_size=max_nodes))

    if not node_datas:
        return graph

    graph.add_nodes_from(enumerate(node_datas))

    # Draw a set of initial edges that guarantee that graph will be connected.
    # We use the invariant that all nodes < n_idx are connected. We create an
    # edge between n_idx and one of those before so that all nodes < n_idx + 1
    # are now connected.
    if connected:
        # Shrink towards high index, so shrink to the path graph. Otherwise
        # it'll shrink to the star graph.
        initial_edges = [draw(st.tuples(st.integers(-(n_idx-1), 0).map(lambda x: -x),
                                        st.just(n_idx)))
                         for n_idx in range(1, len(graph))]
        graph.add_edges_from(initial_edges)

    # Now for the mess. The maximum number of edges possible depends on the
    # graph type.
    if not is_multigraph:
        # Multi(Di)Graphs can make an infinite number of edges. For everything
        # else we clamp the range to (0, max_possible_edges)
        max_possible_edges = len(graph) * (len(graph) - 1)
        if is_directed:
            max_possible_edges *= 2
        if self_loops:
            max_possible_edges += len(graph)
        if max_edges is None or max_edges > max_possible_edges:
            max_edges = max_possible_edges
    if max_edges is not None:
        # Correct for number of edges already made if graph is connected.
        # This may mean we added more edges than originally allowed.
        max_edges -= len(graph.edges)
        if max_edges < 0:
            max_edges = 0

    # Likewise for min_edges
    # We already added some edges, so subtract those.
    min_edges -= len(graph.edges)
    if min_edges < 0:
        min_edges = 0
    elif max_edges is not None and min_edges > max_edges:
        min_edges = max_edges

    def edge_filter(edge):
        """
        Helper function to decide whether the edge between idx and jdx can still
        be added to graph.
        """
        # <= because self loops
        idx, jdx = edge
        return ((not graph.has_edge(idx, jdx) or is_multigraph) and
                (idx <= jdx or is_directed) and
                (idx != jdx or self_loops))

    # We need to sample a number of items from options, these items are
    # possibly not unique. In addition, we need to draw the same number of
    # items from edge_data and associate the two. To top it off, uniqueness
    # is defined by the content of the first element of the tuple.
    edges = st.lists(
        st.tuples(
            st.integers(min_value=0, max_value=len(graph) - 1),
            st.integers(min_value=0, max_value=len(graph) - 1),
        ).filter(edge_filter),
        unique=not is_multigraph,
        min_size=min_edges,
        max_size=max_edges
    )
    graph.add_edges_from(draw(edges))

    edge_datas = draw(st.lists(
        edge_data,
        min_size=len(graph.edges),
        max_size=len(graph.edges)
    ))
    for edge, data in zip(graph.edges, edge_datas):
        graph.edges[edge].update(data)

    if node_keys is not None:
        new_idxs = draw(st.sets(node_keys,
                                min_size=len(graph),
                                max_size=len(graph)))
        graph = nx.relabel_nodes(graph, dict(zip(list(graph), list(new_idxs))))

    return graph

In [ ]:
pareto_optimal_model = cp_model.CpModel()